In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tnrange, tqdm_notebook
import gc

In [2]:
sns.set_context('talk')

In [3]:
import skimage.io as io

In [4]:
DEBUG = True

In [5]:
import theano
theano.config.openmp = True

In [6]:
import keras

Using Theano backend.


# Preprocessing

In [7]:
labels = pd.read_csv('finetuning-train/gt.csv')

In [8]:
labels.head()

,filename,class_id
0,0000.png,50
1,0001.png,24
2,0002.png,25
3,0003.png,37
4,0004.png,13


In [9]:
labels.max()

filename    2499.png
class_id          50
dtype: object

In [10]:
labels.min()

filename    0000.png
class_id           1
dtype: object

Раскидаем train по папочкам-классам

In [13]:
import os

In [14]:
def make_dirs(work_dir):
    for index in range(1, 51):
        os.mkdir(join(work_dir, "{:02d}".format(index)))

In [15]:
make_dirs('finetuning-train/images')

In [16]:
import shutil

In [17]:
for row in labels.itertuples():
    shutil.move(join('finetuning-train/images', row.filename), 
                join('finetuning-train/images', "{:02d}".format(row.class_id), row.filename))

Теперь организуем такую же по структуре папку validation, и переместим туда по 10 случайных фотографий из каждого класса

In [20]:
make_dirs('finetuning-train/validation')

In [19]:
import random

def make_validation(input_dir, output_dir):
    files = random.sample([f for f in listdir(input_dir) if isfile(join(input_dir, f))], 10)
    for file in files:
        shutil.move(join(input_dir, file), join(output_dir, file))

In [21]:
for index in range(1, 51):
    make_validation(join('finetuning-train/images', "{:02d}".format(index)),
                    join('finetuning-train/validation', "{:02d}".format(index)))

Растянем/сожмём изображения, пока меньшая из сторон не станет больше или равна SIDE_SIZE. Из этой картинки вырежем не более 20 раз квадрат размера SIDE_SIZE на SIDE_SIZE, размножив таким образом выборку

In [33]:
SIDE_SIZE = 250

In [35]:
from os import listdir
from os.path import isfile, join
import warnings
import skimage.color
import skimage.transform
import random

In [23]:
make_dirs('finetuning-train/cut250')

In [218]:
def cut(input_dir, output_dir):
    files = sorted([f for f in listdir(input_dir) if isfile(join(input_dir, f))])
    for file in files:
        img = io.imread(join(input_dir, file))
        if img.ndim == 2:
            img = skimage.color.gray2rgb(img)
        factor = max([SIDE_SIZE / img.shape[i] for i in range(2)])
        if factor > 1:
            img = skimage.transform.rescale(img, factor,
                                               mode='reflect', preserve_range=True)
        iter_num = int(((img.shape[0] - 250) / 10 + 1) * ((img.shape[1] - 250) / 10 + 1))
        for index in range(min(20, iter_num)):
            side_size = random.randint(SIDE_SIZE, min(img.shape[:-1]))
            first = random.randrange(img.shape[0] - side_size + 1)
            second = random.randrange(img.shape[1] - side_size + 1)
            sqr_img = img[first : first + side_size,
                          second : second + side_size]
            if side_size > SIDE_SIZE:
                sqr_img = skimage.transform.rescale(sqr_img, SIDE_SIZE / side_size,
                                                    mode='reflect', preserve_range=True)
            sqr_img = sqr_img.astype(np.uint8)    
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                io.imsave(join(output_dir, "{}_{}".format(index, file)), sqr_img)

In [219]:
%%time
cut('finetuning-train/images/01', 'finetuning-train/cut250/01')

CPU times: user 58.9 s, sys: 56.6 s, total: 1min 55s
Wall time: 39.7 s


In [220]:
%%time
for index in range(2, 51):
    cut(join('finetuning-train/images', "{:02d}".format(index)),
        join('finetuning-train/cut250', "{:02d}".format(index)))

CPU times: user 1h 9min 16s, sys: 1h 4min 41s, total: 2h 13min 57s
Wall time: 48min 1s


In [221]:
make_dirs('finetuning-train/validation_cut250')

In [222]:
%%time
for index in range(1, 51):
    cut(join('finetuning-train/validation', "{:02d}".format(index)),
        join('finetuning-train/validation_cut250', "{:02d}".format(index)))

CPU times: user 15min 31s, sys: 15min 14s, total: 30min 46s
Wall time: 10min 27s


Тестовую выборку тоже размножим

In [225]:
for index in range(500):
    os.mkdir(join('finetuning-test/cut250', "{:04d}".format(index)))

In [226]:
for index in range(500):
    os.mkdir(join('finetuning-test/img_subdirs', "{:04d}".format(index)))

In [227]:
for index in range(500):
    shutil.copy(join('finetuning-test/images', "{:04d}.png".format(index)), 
                join('finetuning-test/img_subdirs', "{:04d}".format(index), "{:04d}.png".format(index)))

In [228]:
%%time
for index in range(500):
    cut(join('finetuning-test/img_subdirs', "{:04d}".format(index)),
        join('finetuning-test/cut250', "{:04d}".format(index)))

CPU times: user 16min 24s, sys: 15min 59s, total: 32min 24s
Wall time: 11min 1s


In [263]:
%%time

for index in range(500):
    src = join('finetuning-test/cut250', "{:04d}".format(index))
    dst = join(src, 'dir')
    files = sorted([f for f in listdir(src) if isfile(join(src, f))])
    os.mkdir(dst)
    for file in files:
        shutil.move(join(src, file), join(dst, file))

CPU times: user 336 ms, sys: 256 ms, total: 592 ms
Wall time: 872 ms


Нам понадобятся DataGenerator'ы для обучения и валидации.

In [230]:
from keras.preprocessing.image import ImageDataGenerator

In [231]:
data_gen_args = dict(rotation_range=10.,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.2,
                     fill_mode='reflect',
                     channel_shift_range=0.1,
                     horizontal_flip=True,
                     vertical_flip=False,)

Напишем обёртку, которая будет вызывать prepocess_input для картинок из генератора

In [232]:
from keras.applications.resnet50 import preprocess_input

def preprocessed_datagen(datagen):
    for x, y in datagen:
        yield preprocess_input(x), y

In [233]:
batch_size=100

train_gen = preprocessed_datagen(ImageDataGenerator(**data_gen_args).flow_from_directory(
            'finetuning-train/cut250', target_size=(SIDE_SIZE, SIDE_SIZE), class_mode='categorical',
             batch_size=batch_size, shuffle=True, seed=10))

validation_gen = preprocessed_datagen(ImageDataGenerator(**data_gen_args).flow_from_directory(
                 'finetuning-train/validation_cut250', target_size=(SIDE_SIZE, SIDE_SIZE), class_mode='categorical',
                 batch_size=batch_size, shuffle=True, seed=10))

Found 39708 images belonging to 50 classes.
Found 9968 images belonging to 50 classes.


# Model

Будем использовать ResNet50. Для начала снимем полносвязные слои

In [234]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import Dense
from keras.layers import Input

base_model = ResNet50(weights='imagenet', include_top=False, 
                         input_tensor=Input(shape=(SIDE_SIZE, SIDE_SIZE, 3)))

(Subtensor{int64}.0, Elemwise{add,no_inplace}.0, Elemwise{add,no_inplace}.0, Subtensor{int64}.0)


In [235]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

0 input_1
1 zeropadding2d_1
2 conv1
3 bn_conv1
4 activation_1
5 maxpooling2d_1
6 res2a_branch2a
7 bn2a_branch2a
8 activation_2
9 res2a_branch2b
10 bn2a_branch2b
11 activation_3
12 res2a_branch2c
13 res2a_branch1
14 bn2a_branch2c
15 bn2a_branch1
16 merge_1
17 activation_4
18 res2b_branch2a
19 bn2b_branch2a
20 activation_5
21 res2b_branch2b
22 bn2b_branch2b
23 activation_6
24 res2b_branch2c
25 bn2b_branch2c
26 merge_2
27 activation_7
28 res2c_branch2a
29 bn2c_branch2a
30 activation_8
31 res2c_branch2b
32 bn2c_branch2b
33 activation_9
34 res2c_branch2c
35 bn2c_branch2c
36 merge_3
37 activation_10
38 res3a_branch2a
39 bn3a_branch2a
40 activation_11
41 res3a_branch2b
42 bn3a_branch2b
43 activation_12
44 res3a_branch2c
45 res3a_branch1
46 bn3a_branch2c
47 bn3a_branch1
48 merge_4
49 activation_13
50 res3b_branch2a
51 bn3b_branch2a
52 activation_14
53 res3b_branch2b
54 bn3b_branch2b
55 activation_15
56 res3b_branch2c
57 bn3b_branch2c
58 merge_5
59 activation_16
60 res3c_branch2a
61 bn3c_branch

Добавим один выходной полносвязный слой, как в оригинальной ResNet50

In [236]:
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.layers import Dropout
from keras.layers import Flatten, GlobalAveragePooling2D

In [238]:
x = base_model.output

x = Flatten()(x)
predictions = Dense(50, activation='softmax')(x)

model = Model(input=base_model.input, output=predictions)

for layer in base_model.layers:
    layer.trainable = False
    
model.compile(#optimizer=SGD(lr=lr, decay=decay, nesterov=True, momentum=0.9),
              optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy', 'categorical_crossentropy'])

# Learning

In [239]:
%%time

hist = model.fit_generator(
        train_gen,
        samples_per_epoch=3000,
        nb_epoch=5,
        validation_data=validation_gen,
        nb_val_samples=600,
        )

Epoch 1/5
3000/3000 [==============================] - 4712s - loss: 2.7038 - categorical_accuracy: 0.3320 - categorical_crossentropy: 2.7038 - val_loss: 1.9756 - val_categorical_accuracy: 0.4550 - val_categorical_crossentropy: 1.9756
Epoch 2/5
3000/3000 [==============================] - 4456s - loss: 1.3947 - categorical_accuracy: 0.6353 - categorical_crossentropy: 1.3947 - val_loss: 1.4060 - val_categorical_accuracy: 0.5800 - val_categorical_crossentropy: 1.4060
Epoch 3/5
3000/3000 [==============================] - 4757s - loss: 0.9410 - categorical_accuracy: 0.7547 - categorical_crossentropy: 0.9410 - val_loss: 1.3002 - val_categorical_accuracy: 0.6317 - val_categorical_crossentropy: 1.3002
Epoch 4/5
3000/3000 [==============================] - 4803s - loss: 0.7618 - categorical_accuracy: 0.7933 - categorical_crossentropy: 0.7618 - val_loss: 1.0268 - val_categorical_accuracy: 0.7033 - val_categorical_crossentropy: 1.0268
Epoch 5/5
3000/3000 [==============================] - 4929s

In [240]:
model.save('cut_5epoch_top_resnet.h5')

In [241]:
from keras.models import load_model

In [242]:
model = load_model('cut_5epoch_top_resnet.h5')

(Subtensor{int64}.0, Elemwise{add,no_inplace}.0, Elemwise{add,no_inplace}.0, Subtensor{int64}.0)


Дообучим некоторые последние res-блоки

Сначала разморозим эти блоки

In [243]:
for layer in model.layers[:142]:
   layer.trainable = False
for layer in model.layers[142:]:
   layer.trainable = True

Возьмём RMSprop с меньшим learning rate (по умолчанию lr=$10^{-3}$)

In [244]:
gc.collect()

247

In [245]:
from keras.optimizers import RMSprop

In [246]:
lr=1 * 10 ** -4

In [247]:
model.compile(optimizer=RMSprop(lr=lr),
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy', 'categorical_crossentropy'])

В data-генераторах поменяем seed

In [248]:
batch_size=100

train_gen = preprocessed_datagen(ImageDataGenerator(**data_gen_args).flow_from_directory(
            'finetuning-train/cut250', target_size=(SIDE_SIZE, SIDE_SIZE), class_mode='categorical',
             batch_size=batch_size, shuffle=True, seed=42))

validation_gen = preprocessed_datagen(ImageDataGenerator(**data_gen_args).flow_from_directory(
                 'finetuning-train/validation_cut250', target_size=(SIDE_SIZE, SIDE_SIZE), class_mode='categorical',
                 batch_size=batch_size, shuffle=True, seed=42))

Found 39708 images belonging to 50 classes.
Found 9968 images belonging to 50 classes.


In [249]:
%%time

hist = model.fit_generator(
        train_gen,
        samples_per_epoch=3000,
        nb_epoch=2,
        validation_data=validation_gen,
        nb_val_samples=600,
        )

Epoch 1/2
3000/3000 [==============================] - 5599s - loss: 0.3531 - categorical_accuracy: 0.9090 - categorical_crossentropy: 0.3531 - val_loss: 0.9303 - val_categorical_accuracy: 0.7450 - val_categorical_crossentropy: 0.9303
Epoch 2/2
3000/3000 [==============================] - 5903s - loss: 0.2339 - categorical_accuracy: 0.9367 - categorical_crossentropy: 0.2339 - val_loss: 0.9554 - val_categorical_accuracy: 0.7467 - val_categorical_crossentropy: 0.9554
CPU times: user 7h 16min 2s, sys: 1h 7s, total: 8h 16min 10s
Wall time: 3h 14min 52s


In [250]:
model.save('cut_2epoch_top_res_block_resnet.h5')

In [251]:
model = load_model('cut_2epoch_top_res_block_resnet.h5')

(Subtensor{int64}.0, Elemwise{add,no_inplace}.0, Elemwise{add,no_inplace}.0, Subtensor{int64}.0)


Уменьшим lr

In [252]:
lr=1 * 10 ** -5

In [253]:
model.compile(optimizer=RMSprop(lr=lr),
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy', 'categorical_crossentropy'])

In [254]:
batch_size=100

train_gen = preprocessed_datagen(ImageDataGenerator(**data_gen_args).flow_from_directory(
            'finetuning-train/cut250', target_size=(SIDE_SIZE, SIDE_SIZE), class_mode='categorical',
             batch_size=batch_size, shuffle=True, seed=67))

validation_gen = preprocessed_datagen(ImageDataGenerator(**data_gen_args).flow_from_directory(
                 'finetuning-train/validation_cut250', target_size=(SIDE_SIZE, SIDE_SIZE), class_mode='categorical',
                 batch_size=batch_size, shuffle=True, seed=67))

Found 39708 images belonging to 50 classes.
Found 9968 images belonging to 50 classes.


In [255]:
%%time

hist = model.fit_generator(
        train_gen,
        samples_per_epoch=3000,
        nb_epoch=2,
        validation_data=validation_gen,
        nb_val_samples=600,
        )

Epoch 1/2
3000/3000 [==============================] - 5753s - loss: 0.1723 - categorical_accuracy: 0.9600 - categorical_crossentropy: 0.1723 - val_loss: 0.7677 - val_categorical_accuracy: 0.7800 - val_categorical_crossentropy: 0.7677
Epoch 2/2
3000/3000 [==============================] - 5897s - loss: 0.1443 - categorical_accuracy: 0.9697 - categorical_crossentropy: 0.1443 - val_loss: 0.6824 - val_categorical_accuracy: 0.8083 - val_categorical_crossentropy: 0.6824
CPU times: user 7h 24min 2s, sys: 1h 46s, total: 8h 24min 48s
Wall time: 3h 17min 33s


In [256]:
model.save('cut_4epoch_top_res_block_resnet.h5')

In [257]:
model = load_model('cut_4epoch_top_res_block_resnet.h5')

(Subtensor{int64}.0, Elemwise{add,no_inplace}.0, Elemwise{add,no_inplace}.0, Subtensor{int64}.0)


Уменьшим lr

In [258]:
lr=1 * 10 ** -6

In [259]:
model.compile(optimizer=RMSprop(lr=lr),
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy', 'categorical_crossentropy'])

In [260]:
batch_size=100

train_gen = preprocessed_datagen(ImageDataGenerator(**data_gen_args).flow_from_directory(
            'finetuning-train/cut250', target_size=(SIDE_SIZE, SIDE_SIZE), class_mode='categorical',
             batch_size=batch_size, shuffle=True, seed=95))

validation_gen = preprocessed_datagen(ImageDataGenerator(**data_gen_args).flow_from_directory(
                 'finetuning-train/validation_cut250', target_size=(SIDE_SIZE, SIDE_SIZE), class_mode='categorical',
                 batch_size=batch_size, shuffle=True, seed=95))

Found 39708 images belonging to 50 classes.
Found 9968 images belonging to 50 classes.


In [261]:
%%time

hist = model.fit_generator(
        train_gen,
        samples_per_epoch=3000,
        nb_epoch=2,
        validation_data=validation_gen,
        nb_val_samples=600,
        )

Epoch 1/2
3000/3000 [==============================] - 5271s - loss: 0.1511 - categorical_accuracy: 0.9663 - categorical_crossentropy: 0.1511 - val_loss: 0.7874 - val_categorical_accuracy: 0.7783 - val_categorical_crossentropy: 0.7874
Epoch 2/2
3000/3000 [==============================] - 5538s - loss: 0.1600 - categorical_accuracy: 0.9673 - categorical_crossentropy: 0.1600 - val_loss: 0.7560 - val_categorical_accuracy: 0.7700 - val_categorical_crossentropy: 0.7560
CPU times: user 6h 49min 41s, sys: 59min 7s, total: 7h 48min 48s
Wall time: 3h 3min 7s


In [262]:
model.save('cut_6epoch_top_res_block_resnet.h5')

# Result

Возьмём версию модели, которая получила наилучший результат на валидации

In [264]:
model = load_model('cut_4epoch_top_res_block_resnet.h5')

(Subtensor{int64}.0, Elemwise{add,no_inplace}.0, Elemwise{add,no_inplace}.0, Subtensor{int64}.0)


Результатом для экземпляра из тестовой выборки будет усреднённая выдача нейросети на квадратах, полученных из этого экземпляра

In [265]:
def test_preprocessed_datagen(datagen):
    for x in datagen:
        yield preprocess_input(x)

In [266]:
TEST_SAMPLES_COUNT = 500
predictions = np.zeros(TEST_SAMPLES_COUNT)

In [268]:
%%time

for index in range(TEST_SAMPLES_COUNT):
    src = join('finetuning-test/cut250', "{:04d}".format(index))
    dst = join(src, 'dir')
    val_samples = len([f for f in listdir(dst) if isfile(join(dst, f))])
    test_gen = test_preprocessed_datagen(ImageDataGenerator().flow_from_directory(
                 src, target_size=(SIDE_SIZE, SIDE_SIZE), class_mode=None,
                 batch_size=val_samples, shuffle=False, seed=10))
    probs = model.predict_generator(test_gen, val_samples=val_samples)
    probs = np.mean(probs, axis=0)
    predictions[index] = np.argmax(probs) + 1

Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.


Сохраним предсказания в файл

In [269]:
df_preds = pd.DataFrame()
df_preds['filename'] = ["{:04d}.png".format(index) for index in range(TEST_SAMPLES_COUNT)]
df_preds['class_id'] = predictions

df_preds.to_csv("csv/cut_dense5_topres4.csv", index=False)